# Create an interface with minimal strain and relax it using ASE EMT

Use Zur and McGill superlattices matching [algorithm](https://doi.org/10.1063/1.3330840) to create interfaces between two materials using the Pymatgen [implementation](https://pymatgen.org/pymatgen.analysis.interfaces.html#pymatgen.analysis.interfaces.zsl). And then relax the interface using the EMT (Effective Medium Theory) potential. 
> <b style="color:red">NOTE:</b> The [EMT potential](https://wiki.fysik.dtu.dk/ase/ase/calculators/emt.html) is available for a limited number of elements (Al, Cu, Ag, Au, Ni, Pd and Pt., as well as H, C, N, O in a limited way). If the interface contains elements not supported by EMT, the relaxation will not be performed.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials
2. Set Input Parameters (e.g. `THICKNESS`, `MAX_AREA`, `FMAX`) below or use the default values
3. Click "Run" > "Run All" to run all cells
4. Wait for the run to complete (depending on the parameters it can take a few min or more). Scroll down to view cell results.
5. Review the strain plot and modify its parameters as needed
6. Apply relaxation to the selected interface and analyze the difference
7. Pass the results back to the web application

## Methodology

The following happens in the script below:

1. The steps from [create_interface_with_min_strain_zsl.ipynb](create_interface_with_min_strain_zsl.ipynb)

2. The interface with the lowest strain is selected and relaxed using the optimizer selected below (BFGS, by default). The EMT potential is used as an energy calculator.


## 1. Set Input Parameters

### 1.1. Select Substrate and Layer from Input Materials


In [ ]:
SUBSTRATE_PARAMETERS = {
    "MATERIAL_INDEX": 0,  # the index of the material in the materials_in list
    "MILLER_INDICES": (1, 1, 1),  # the miller indices of the interfacial plane
    "THICKNESS": 3,  # in layers
}

LAYER_PARAMETERS = {
    "MATERIAL_INDEX": 1,  # the index of the material in the materials_in list
    "MILLER_INDICES": (0, 0, 1),  # the miller indices of the interfacial plane
    "THICKNESS": 1,  # in layers
}

USE_CONVENTIONAL_CELL = True  # if True, the surface plane is constructed using miller indices of the conventional cell

### 1.2. Set Interface Parameters

The distance between layer and substrate and maximum area to consider when matching.


In [ ]:
INTERFACE_PARAMETERS = {
    "DISTANCE_Z": 3.0,  # in Angstroms
    "MAX_AREA": 400,  # in Angstroms^2
}

### 1.3. Set Algorithm Parameters


In [ ]:
ZSL_PARAMETERS = {
    "MAX_AREA": INTERFACE_PARAMETERS["MAX_AREA"],  # The area to consider in Angstrom^2
    "MAX_AREA_TOL": 0.09,  # The area within this tolerance is considered equal
    "MAX_LENGTH_TOL": 0.03,  # supercell lattice vectors lengths within this tolerance are considered equal
    "MAX_ANGLE_TOL": 0.01,  # supercell lattice angles within this tolerance are considered equal
    "STRAIN_TOL": 10e-6,  # strains within this tolerance are considered equal
}
RELAXATION_PARAMETERS = {
    # Relaxation stops when the largest force component is less than fmax.
    # In ev/Angstrom, per https://wiki.fysik.dtu.dk/ase/ase/optimize.html
    "FMAX": 0.05,
    # The optimization algorithm: BFGS, FIRE, etc.
    # per https://wiki.fysik.dtu.dk/ase/ase/optimize.html#local-optimization
    "OPTIMIZER": "BFGS",
}

## 2. Install Packages


In [ ]:
import sys
if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
from utils.jupyterlite import install_packages
await install_packages("create_interface_with_min_strain_zsl.ipynb","../../config.yml")

## 3. Load and prepare Input Materials

In [ ]:
from utils.jupyterlite import get_data
from pymatgen.analysis.structure_analyzer import SpacegroupAnalyzer
from src.utils import to_pymatgen

# Get the list of input materials and load them into `materials_in` variable
get_data("materials_in", globals())

if "materials_in" in globals():
    pymatgen_materials = [to_pymatgen(item) for item in materials_in]
    if USE_CONVENTIONAL_CELL: pymatgen_materials = [SpacegroupAnalyzer(item).get_conventional_standard_structure() for
                                                    item in pymatgen_materials]

    for material in pymatgen_materials:
        print(material, "\n")

## 4. Create interfaces

### 4.1. Extract Interfaces and Terminations

Extract all possible layer/substrate supercell combinations within the maximum area including different terminations.


In [ ]:
from src.pymatgen_coherent_interface_builder import CoherentInterfaceBuilder, ZSLGenerator
from src.utils import translate_to_bottom

# Translate the materials to the bottom of the cell to allow for multilayer heterostructures creation
pymatgen_materials = [translate_to_bottom(item) for item in pymatgen_materials]

def create_interfaces(settings: dict):
    print("Creating interfaces...")
    zsl = ZSLGenerator(
        max_area_ratio_tol=settings["ZSL_PARAMETERS"]["MAX_AREA_TOL"],
        max_area=settings["ZSL_PARAMETERS"]["MAX_AREA"],
        max_length_tol=settings["ZSL_PARAMETERS"]["MAX_LENGTH_TOL"],
        max_angle_tol=settings["ZSL_PARAMETERS"]["MAX_ANGLE_TOL"],
    )

    cib = CoherentInterfaceBuilder(
        substrate_structure=pymatgen_materials[settings["SUBSTRATE_PARAMETERS"]["MATERIAL_INDEX"]],
        film_structure=pymatgen_materials[settings["LAYER_PARAMETERS"]["MATERIAL_INDEX"]],
        substrate_miller=settings["SUBSTRATE_PARAMETERS"]["MILLER_INDICES"],
        film_miller=settings["LAYER_PARAMETERS"]["MILLER_INDICES"],
        zslgen=zsl,
        strain_tol=settings["ZSL_PARAMETERS"]["STRAIN_TOL"],
    )

    # Find terminations
    cib._find_terminations()
    terminations = cib.terminations

    # Create interfaces for each termination
    interfaces = {}
    for termination in terminations:
        interfaces[termination] = []
        for interface in cib.get_interfaces(
                termination,
                gap=settings["INTERFACE_PARAMETERS"]["DISTANCE_Z"],
                film_thickness=settings["LAYER_PARAMETERS"]["THICKNESS"],
                substrate_thickness=settings["SUBSTRATE_PARAMETERS"]["THICKNESS"],
                in_layers=True,
        ):
            # Wrap atoms to unit cell
            interface["interface"].make_supercell((1, 1, 1), to_unit_cell=True)
            interfaces[termination].append(interface)
    return interfaces, terminations


interfaces, terminations = create_interfaces(
    settings={
        "SUBSTRATE_PARAMETERS": SUBSTRATE_PARAMETERS,
        "LAYER_PARAMETERS": LAYER_PARAMETERS,
        "ZSL_PARAMETERS": ZSL_PARAMETERS,
        "INTERFACE_PARAMETERS": INTERFACE_PARAMETERS,
    }
)


### 4.2. Print out the interfaces and terminations


In [ ]:
print(f'Found {len(terminations)} terminations')
for termination in terminations:
    print(f"Found {len(interfaces[termination])} interfaces for", termination, "termination")

## 5. Sort interfaces by strain

### 5.1. Sort all interfaces


In [ ]:
# Could be "strain", "von_mises_strain", "mean_abs_strain"
strain_mode = "mean_abs_strain"


# Sort interfaces by the specified strain mode and number of sites
def sort_interfaces(interfaces, terminations):
    sorted_interfaces = {}
    for termination in terminations:
        sorted_interfaces[termination] = sorted(
            interfaces[termination], key=lambda x: (x[strain_mode], x["interface"].num_sites)
        )
    return sorted_interfaces


sorted_interfaces = sort_interfaces(interfaces, terminations)

### 5.2. Print out interfaces with lowest strain for each termination


In [ ]:
for termination in terminations:
    print(f"Interface with lowest strain for termination {termination} (index 0):")
    first_interface = interfaces[termination][0]
    print("    strain:", first_interface[strain_mode] * 100, "%")
    print("    number of atoms:", first_interface["interface"].num_sites)

## 6. Plot the results

Plot the number of atoms vs strain. Adjust the parameters as needed.


In [ ]:
import plotly.graph_objs as go
from collections import defaultdict

PLOT_SETTINGS = {
    "HEIGHT": 600,
    "X_SCALE": "log",  # or linear
    "Y_SCALE": "log",  # or linear
}


def plot_strain_vs_atoms(sorted_interfaces, terminations, settings):
    # Create a mapping from termination to its index
    termination_to_index = {termination: i for i, termination in enumerate(terminations)}

    grouped_interfaces = defaultdict(list)
    for termination, interfaces in sorted_interfaces.items():
        for index, interface_data in enumerate(interfaces):
            strain_percentage = interface_data["mean_abs_strain"] * 100
            num_sites = interface_data["interface"].num_sites
            key = (strain_percentage, num_sites)
            grouped_interfaces[key].append((index, termination))

    data = []
    for (strain, num_sites), indices_and_terminations in grouped_interfaces.items():
        termination_indices = defaultdict(list)
        for index, termination in indices_and_terminations:
            termination_indices[termination].append(index)
        all_indices = [index for indices in termination_indices.values() for index in indices]
        index_range = f"{min(all_indices)}-{max(all_indices)}" if len(all_indices) > 1 else str(min(all_indices))

        hover_text = "<br>-----<br>".join(
            f"Termination: {termination}<br>Termination index: {termination_to_index[termination]}<br>Interfaces Index Range: {index_range}<br>Strain: {strain:.2f}%<br>Atoms: {num_sites}"
            for termination, indices in termination_indices.items()
        )
        trace = go.Scatter(
            x=[strain],
            y=[num_sites],
            text=[hover_text],
            mode="markers",
            hoverinfo="text",
            name=f"Indices: {index_range}",
        )
        data.append(trace)

    layout = go.Layout(
        xaxis=dict(title="Strain (%)", type=settings["X_SCALE"]),
        yaxis=dict(title="Number of atoms", type=settings["Y_SCALE"]),
        hovermode="closest",
        height=settings["HEIGHT"],
        legend_title_text="Interfaces Index Range",
    )
    fig = go.Figure(data=data, layout=layout)
    fig.show()


plot_strain_vs_atoms(sorted_interfaces, terminations, PLOT_SETTINGS)

for i, termination in enumerate(terminations):
    print(f"Termination {i}:", termination)

## 7. Select the interface with the desired termination and strain

The data in `sorted_interfaces` now contains an object with the following structure:

```json
{
    "('C_P6/mmm_2', 'Si_R-3m_1')": [
        { ...interface for ('C_P6/mmm_2', 'Si_R-3m_1') at index 0...},
        { ...interface for ('C_P6/mmm_2', 'Si_R-3m_1') at index 1...},
        ...
    ],
    "<termination at index 1>": [
        { ...interface for 'termination at index 1' at index 0...},
        { ...interface for 'termination at index 1' at index 1...},
        ...
    ]
}
```

Select the index for termination first, and for it - the index in the list of corresponding interfaces sorted by strain (index 0 has minimum strain).


In [ ]:
termination_index = 0
interface_index = 0

termination = terminations[termination_index]

interface = sorted_interfaces[termination][interface_index]["interface"]

interface_strain = f"{sorted_interfaces[termination][interface_index]['mean_abs_strain'] * 100:.2f}%"

## 8. Apply relaxation to the interface

### 8.1. Apply relaxation to the selected interface with ASE

Optimizer is set from the available options in the settings and EMT is used as the energy calculator.

In [ ]:
import logging

import plotly.graph_objs as go
from IPython.display import display
from plotly.subplots import make_subplots

# Per https://github.com/materialsvirtuallab/matgl/blob/main/examples/Relaxations%20and%20Simulations%20using%20the%20M3GNet%20Universal%20Potential.ipynb
from src.utils import poscar_to_ase, ase_to_poscar, ase_to_pymatgen, pymatgen_to_ase
from ase.optimize import BFGS, FIRE
from ase.calculators.emt import EMT, parameters as EMT_parameters

calculator = EMT()
ase_original_interface = pymatgen_to_ase(interface)

# Create a plotly figure widget to display energy convergence
fig = make_subplots(rows=1, cols=1, specs=[[{"type": "scatter"}]])
scatter = go.Scatter(x=[], y=[], mode='lines+markers', name='Energy')
fig.add_trace(scatter)
fig.update_layout(title_text='Convergence', xaxis_title='Step', yaxis_title='Energy (eV)')
# Display figure widget
f = go.FigureWidget(fig)
display(f)


# Define a callback function to update the plot at each step
def plotly_callback():
    step = dyn.nsteps
    energy = ase_interface.get_total_energy()

    # Add the new step and energy to the lists
    steps.append(step)
    energies.append(energy)

    print(f"Step: {step}, Energy: {energy:.4f} eV")

    # Update the figure with the new data
    with f.batch_update():
        f.data[0].x = steps
        f.data[0].y = energies


# check if EMT potential is available for every element in the interface
emt_elements = EMT_parameters.keys()
unique_elements = set(site.species_string for site in interface.sites)
if unique_elements.issubset(emt_elements):
    # select optimizer based on the setup parameters
    optimizer = {"BFGS": BFGS, "FIRE": FIRE}[RELAXATION_PARAMETERS["OPTIMIZER"]]

    # relax the interface in place
    ase_interface = ase_original_interface.copy()
    ase_interface.set_calculator(calculator)
    dyn = optimizer(ase_interface)
    steps = []
    energies = []

    dyn.attach(plotly_callback, interval=1)
    dyn.run(fmax=RELAXATION_PARAMETERS["FMAX"])

    # extract results
    ase_final_interface = ase_interface
    name_relaxation_suffix = f"Relaxed {RELAXATION_PARAMETERS['OPTIMIZER']} fmax={RELAXATION_PARAMETERS['FMAX']}"
    ase_original_interface.set_calculator(calculator)
    original_energy = ase_original_interface.get_total_energy()
    relaxed_energy = ase_final_interface.get_total_energy()

    # print out the final relaxed structure and energy
    print('Original structure:\n', ase_to_poscar(ase_original_interface))
    print('\nRelaxed structure:\n', ase_to_poscar(ase_final_interface))
    print(f"The final energy is {float(relaxed_energy):.3f} eV.")

else:
    logging.warning(
        f"The EMT potential is not implemented for the following elements in the interface: {unique_elements - emt_elements}.\nList of supported elements: {list(emt_elements)}.")
    ase_final_interface = ase_original_interface
    name_relaxation_suffix = "Non-relaxed"
    relaxed_energy = original_energy = None


### 8.2. View structure before and after relaxation


In [ ]:
import base64
from ase.io import write
from ase.build import make_supercell
from IPython.display import HTML
import io
from src.utils import calculate_average_interlayer_distance


def visualize_material_base64(material, title: str, rotation: str = '0x', number_of_repetitions: int = 3):
    """
    Returns an HTML string with a Base64-encoded image for visualization,
    including the name of the file, positioned horizontally.
    """
    # Set the number of unit cell repetition for the structure
    n = number_of_repetitions
    material_repeat = make_supercell(material, [[n, 0, 0], [0, n, 0], [0, 0, 1]])
    text = f"{material.symbols} - {title}"

    # Write image to a buffer to display in HTML
    buf = io.BytesIO()
    write(buf, material_repeat, format='png', rotation=rotation)
    buf.seek(0)
    img_str = base64.b64encode(buf.read()).decode('utf-8')
    html_str = f'''
    <div style="display: inline-block; margin: 10px; vertical-align: top;">
        <p>{text}</p>
        <img src="data:image/png;base64,{img_str}" alt="{title}" />
    </div>
    '''
    return html_str


html_original = visualize_material_base64(ase_original_interface, "original", "-90x")
html_relaxed = visualize_material_base64(ase_final_interface, "relaxed", "-90x")

# Display the interfaces before and after relaxation
html_content = f'<div style="display: flex;">{html_original}{html_relaxed}</div>'
display(HTML(html_content))

# Calculate the average interlayer distance, see References [1] and [2] below
average_distance = calculate_average_interlayer_distance(ase_final_interface, SUBSTRATE_PARAMETERS["MATERIAL_INDEX"],
                                                         LAYER_PARAMETERS["MATERIAL_INDEX"])
print("Interfacial distance:")
print(f"     Original: {INTERFACE_PARAMETERS['DISTANCE_Z']:.3f} Å")
print(f"Final average: {average_distance:.3f} Å")
print(f"        Delta: {average_distance - INTERFACE_PARAMETERS['DISTANCE_Z']:.3f} Å")

### 8.3. Calculate the energy metrics
Calculate the energy metrics for the relaxed interface.
The effective delta energy per area calculation accounts for the energy contribution of each component (substrate and layer) relative to their proportion in the overall interface. 


In [ ]:
import numpy as np
from ase import Atoms
from ase.calculators.calculator import Calculator


def filter_atoms_by_tag(atoms: Atoms, material_index):
    """Filter atoms by their tag, corresponding to the material index."""
    return atoms[atoms.get_tags() == material_index]


def get_surface_area(atoms: Atoms):
    """Calculate the surface area of the atoms."""
    matrix = atoms.cell
    return np.linalg.norm(np.cross(matrix[0], matrix[1]))


def get_total_energy(atoms: Atoms, calculator: Calculator):
    """Set calculator for atoms and return their total energy."""
    atoms.set_calculator(calculator)
    return atoms.get_total_energy()


def get_total_energy_per_atom(atoms: Atoms, calculator: Calculator):
    """Calculate the energy per atom."""
    return get_total_energy(atoms, calculator) / atoms.get_global_number_of_atoms()


def get_surface_energy(slab: Atoms, bulk: Atoms, calculator: Calculator):
    """Calculate the surface energy by subtracting the bulk energy from the slab energy."""
    number_of_atoms = slab.get_global_number_of_atoms()
    area = get_surface_area(slab)
    return (get_total_energy(slab, calculator) - get_total_energy_per_atom(bulk, calculator) * number_of_atoms) / (
            2 * area)


def get_adhesion_energy(interface: Atoms, substrate_slab: Atoms, layer_slab: Atoms, calculator: Calculator):
    """Calculate the adhesion energy.
    The adhesion energy is the difference between the energy of the interface and the sum of the energies of the substrate and layer.
    According to: 10.1088/0953-8984/27/30/305004
    """
    energy_substrate_slab = get_total_energy(substrate_slab, calculator)
    energy_layer_slab = get_total_energy(layer_slab, calculator)
    energy_interface = get_total_energy(interface, calculator)
    area = get_surface_area(interface)
    return (energy_substrate_slab + energy_layer_slab - energy_interface) / area


def get_interfacial_energy(interface: Atoms, substrate_slab: Atoms, substrate_bulk: Atoms, layer_slab: Atoms,
                           layer_bulk: Atoms, calculator: Calculator):
    """Calculate the interfacial energy.
    The interfacial energy is the sum of the surface energies of the substrate and layer minus the adhesion energy.
    According to Dupré's formula"""

    surface_energy_substrate = get_surface_energy(substrate_slab, substrate_bulk, calculator)
    surface_energy_layer = get_surface_energy(layer_slab, layer_bulk, calculator)
    adhesion_energy = get_adhesion_energy(interface, substrate_slab, layer_slab, calculator)
    return surface_energy_layer + surface_energy_substrate - adhesion_energy


# Create necessary structures
original_substrate_slab = filter_atoms_by_tag(ase_original_interface, SUBSTRATE_PARAMETERS["MATERIAL_INDEX"])
original_substrate_bulk = pymatgen_to_ase(pymatgen_materials[SUBSTRATE_PARAMETERS["MATERIAL_INDEX"]])
original_layer_slab = filter_atoms_by_tag(ase_original_interface, LAYER_PARAMETERS["MATERIAL_INDEX"])
original_layer_bulk = pymatgen_to_ase(pymatgen_materials[LAYER_PARAMETERS["MATERIAL_INDEX"]])

# Calculate the energy metrics
surface_energy_substrate = get_surface_energy(original_substrate_slab, original_substrate_bulk, calculator)
surface_energy_layer = get_surface_energy(original_layer_slab, original_layer_bulk, calculator)
adhesion_energy = get_adhesion_energy(ase_final_interface, original_substrate_slab, original_layer_slab, calculator)
interfacial_energy = surface_energy_layer + surface_energy_substrate - adhesion_energy

print("Energy metrics:")
print(
    f"Original surface energy substrate: {surface_energy_substrate:.3f} eV/Å^2 ({surface_energy_substrate / 0.16:.3f} J/m^2)")
print(f"Original surface energy layer: {surface_energy_layer:.3f} eV/Å^2 ({surface_energy_layer / 0.16:.3f} J/m^2)")
print(f"Adhesion energy: {adhesion_energy:.3f} eV/Å^2 ({adhesion_energy / 0.16:.3f} J/m^2)")
print(f"Interfacial energy: {interfacial_energy:.3f} eV/Å^2 ({interfacial_energy / 0.16:.3f} J/m^2)")

## 9. Pass relaxed interface to Materials Designer

In [ ]:
from utils.jupyterlite import set_data
from src.utils import from_pymatgen

esse_final_interface = from_pymatgen(ase_to_pymatgen(ase_final_interface))
esse_final_interface[
    'name'] = f"{esse_final_interface['name']}, Interface, Strain: {interface_strain}, {name_relaxation_suffix}"

materials_out = [esse_final_interface]
set_data("materials", materials_out)

## References

[1] Tesch, J., Leicht, P., Blumenschein, F. et al., "Structural and electronic properties of graphene nanoflakes on Au(111) and Ag(111)." Sci Rep 6, 23439 (2016). (https://doi.org/10.1038/srep23439). *Summary*: has Graphene/Au(111) and Graphene/Ag(111) interfaces with distances of 3.23 A and 3.13 A respectively.

[2] Dahal, A., Batzill M., "Graphene–nickel interfaces: a review." Nanoscale, 2014,6, 2548-2562. (https://doi.org/10.1039/C3NR05279F). *Summary*: has notion of Graphene-nickel interface with distance: 2.1 A.